# Varroa Counter
## 1.  Définition du problème

Les colonies d'abeilles du monde entier sont infestées par un parasite qui s'appelle le varroa destructor.
Ce parasite au nom barbare se fixe sur le corps des abeilles adultes et se nourrit de l'hémolymphe. Les femelles pénètrent aussi dans les cellules operculées pour se reproduire sur les larves, ce qui crée plusieurs générations au sein d'une même cellule.
Le varroa transmet des virus aux abeilles et affaiblit leur système immunitaire.
Si rien n'est entrepris pour stopper leur prolifération, les colonies finissent par s'effondrer durant l'automne ou l'hiver.


<img src="https://github.com/geda/varroacounter/blob/main/varroa_destructor.jpg?raw=1" width="50%">

Une fois la récolte du miel effectuée, les apiculteurs effectuent différents traitements sur les colonies, notamment en utilisant de l'acide formique et de l'acide oxalique.
Une fois le traitement effectué, l'apiculteur dépose une planchette sous la ruche afin d'évaluer le degré d'infestation des colonies. Quelques jours après le traitement, les varroas morts tombent sur le fond de la ruche.
Les varroas ayant une taille de 1 à 2 mm, il devient très difficile de les compter lorsqu'ils sont nombreux. De plus, des résidus de cires d'abeilles tombent également des cadres, ce qui complique encore plus le comptage.

<img src="https://github.com/geda/varroacounter/blob/main/fond_varroas.jpg?raw=1" width="50%">

Les varroas sont les petites formes sombres allongées et arrondies.

L'objectif de ce projet est d'estimer automatiquement le niveau d'infestation par les varroas à partir d'une image. Il ne s'agit pas d'obtenir un décompte exact, notamment lorsque les varroas sont peu nombreux, mais plutôt de fournir un ordre de grandeur fiable en cas d'infestation importante — par exemple, distinguer une image contenant 50 varroas d'une autre en contenant 200. Ce type d'évaluation, difficile et fastidieux à réaliser manuellement, est ainsi automatisé pour faciliter le travail de l'apiculteur.

## 2. Collecte de données
J'aimerais pouvoir simplement faire une photo de la planche complete et donné cette image assez large à un modèle pour l'inférence.
J'ai donc recherché des sets de données sur les varroas et j'en ai trouvé plusieurs disponibles sur https://universe.roboflow.com/. Malheureusement aucun dataset ne correspondait parfaitement à mes besoins:
* Images de varroas sur les abeilles et non sur la planche
* Images d'entraînement trop petites
* Images avec des varroas labellisés mais qui ne ressemblent pas vraiment à des varroas

J'ai donc créé un dataset avec mes propres images et j'ai uploadé le dataset sur roboflow sous le projet suivant: https://app.roboflow.com/varroa-counter/varroa-counter-large/2

### Inspection des données
Le dataset de données comprend des
* 32 images d'entraînement (71%)
* 13 images de validation (19%)
* 7 images de test (12%)

## 3. Préparation des Données
J'ai labellisé les 32 images en identifiant plus 1000 varroas. J'ai effectué ce travail très chronophage directement sur roboflow.


Chaque image de ce set a donc maintenant un label associé. Un seul nom de classe est utilisé pour ce dataset: **varroa**
Les labels associés à une image sont sauvegardés au format YOLO et comprennent simplement une suite de nombres comme ceci:
* 0 0.02587890625 0.25439453125 0.0107421875 0.0166015625
* 0 0.021484375 0.27880859375 0.009765625 0.0166015625
* 0 0.0751953125 0.3349609375 0.009765625 0.015625
* ...

#### Explication du format YOLO

```
0 0.02587890625 0.25439453125 0.0107421875 0.0166015625
│      │              │            │            │
│      │              │            │            └── height (hauteur normalisée de la bounding box)
│      │              │            └── width (largeur normalisée de la bounding box)
│      │              └── y_center (position Y du centre, normalisée)
│      └── x_center (position X du centre, normalisée)
└── class_id (identifiant de la classe = 0 = varroa)
```

| Valeur | Signification | Exemple |
|--------|---------------|---------|
| `0` | ID de la classe | varroa (seule classe du dataset) |
| `0.0259` | x_center | Le centre est à 2.6% de la largeur de l'image (très à gauche) |
| `0.2544` | y_center | Le centre est à 25.4% de la hauteur de l'image |
| `0.0107` | width | La box fait 1.07% de la largeur de l'image |
| `0.0166` | height | La box fait 1.66% de la hauteur de l'image |

## 4. Analyse Exploratoire des Données (AED)
Les données contiennent des images avec des fonds de différentes couleurs et matières.

## 5. Feature Engineering
Le modèle devra faire de la détection d'objets. Un des challenges sera de détecter des objets très petits dans les images.
Les 2 features importantes de la détection d'objets seront:
* Classification d'image: déterminer si des varroas sont présents dans l'image
* Localisation d'objet: trouver la position des varroas à l'aide de _bounding boxes_

## 6. Modélisation
Je choisi la classification comme modèle d'apprentissage.

Je divise mon ensemble de donnée comme ceci:
* 26 images d'entraînement (80%)
* 4 images de validation (12%)
* 2 images de test (8%)

In [2]:
!git clone https://github.com/geda/varroacounter.git

Cloning into 'varroacounter'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 40 (delta 18), reused 29 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 11.91 MiB | 19.26 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [3]:
!pip install roboflow

import os
from roboflow import Roboflow

try:
    from google.colab import userdata
    api_key = userdata.get('ROBOFLOW_API_KEY')
except ImportError:
    api_key = os.getenv("ROBOFLOW_API_KEY")

rf = Roboflow(api_key=api_key)


project = rf.workspace("varroa-counter").project("varroa-counter-large")
version = project.version(3)
dataset = version.download("yolov11")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 115.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.13.0.90
    Uninstalling opencv-python-headless-4.13.0.90:
      Successfully uninstalled opencv-python-headless-4.13.0.90
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to varroa-counter-large-3 in yolov11:: 100%|██████████| 62/62 [00:00<00:00, 921.71it/s]


## 7. Entraînement du modèle
### 📌 Version de Python recommandée

Pour cet exercice de réseaux de neurones convolutifs (CNN) avec YOLO11n, je vais utiliser **Python 3.12.7**, car c’est l’une des versions les mieux supportées par YOLO.

In [4]:
# Vérifions la version de Python et du chemin de l'exécutable
import sys
print(sys.version)
print(sys.executable)

3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
/usr/bin/python3


In [3]:
!pip install ultralytics
# import the needed librairies
import ultralytics

from ultralytics import YOLO
# entrainement du modèle
# Je créer un nouveau modèle depuis zéro
model = YOLO('yolo11n.pt')

# lancement de l'entraînement avec seulement 1 seul passage et en abaissant le nombre de batch (nombre d'images traitées simultanément).
# le but est de tester si les capacitées de ma machine sont suffisantes pour entraîner mon modèle
results = model.train(data="varroa-counter-large-3\data.yaml", epochs=1, imgsz=6016,
                      max_det=2000, conf=0.1, iou = 0.5)
print (results)


invalid escape sequence '\d'
invalid escape sequence '\d'
invalid escape sequence '\d'


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.12 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.1, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=varroa-counter-large-3\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1

RuntimeError: Dataset 'varroa-counter-large-3\data.yaml' error ❌ 'varroa-counter-large-3\data.yaml' does not exist

Impossible d'entraîner mon modèle sur ma machine, car elle n'a pas de GPU et le kernel crash.
J'ai fait un essai en utilisant google colab, qui permet gratuitement d'entraîner des modèles avec un peu de GPU. Malheureusement j'ai le même problème sur google colab qui m'indique que ma session a planté après avoir utilisé toute la RAM disponible.

Le problème est l'entraînement de modèle avec des images de grandes tailles car cela nécessite beaucoup de mémoire.

### Changement de stratégie: recherche d'un modèle existant
Je décide donc de rechercher un modèle existant pouvant couvrir mes besoins et j'ai trouvé le travail de recherche suivant: https://www.mdpi.com/2077-0472/15/9/969

**Référence:**
> Yániz, J.; Casalongue, M.; Martinez-de-Pison, F.J.; Silvestre, M.A.; Consortium, B.; Santolaria, P.; Divasón, J. *An AI-Based Open-Source Software for Varroa Mite Fall Analysis in Honeybee Colonies*. Agriculture 2025, 15, 969. https://doi.org/10.3390/agriculture15090969

Leurs modèle a été entraîné avec un dataset de 357 images sur plus de 500 epochs. Ils ont également livré un programme écrit en python permettant d'upload ses images et d'effectuer la détection des varroas avec leurs modèle.
Le code source est disponible sous github ainsi que leurs modèle entraîné: https://github.com/jodivaso/varrodetector/blob/main/model/weights/best.pt

En analysant le code j'ai trouvé particulièrement intéressant qu'ils utilisent une taille d'image assez grande de 6016 pixels (https://github.com/jodivaso/varrodetector/blob/main/varroa_mite_gui.py#L2507C42-L2507C46).



## 8. Evaluation du modèle
Je décide de rebalancer toutes les images de mon dataset en set de test et de n'appliquer aucun redimensionnement d'image

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="tEQkmVJiCxGOZMuDLR6d")
project = rf.workspace("varroa-counter").project("varroa-counter-v3")
version = project.version(3)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


Et de lancer le test du modèle avec mon dataset de test comprenant 32 images

In [ ]:
!pip install ultralytics
# import the needed librairies
import ultralytics
from ultralytics import YOLO

# Charger le modèle entraîné
model = YOLO('varroacounter/model_mdpi_3291496/weights/best.pt')

# Effectuer la détection sur l'image
results = model.val(
    data="varroa-counter-large-3/data.yaml",
    split='train',  # or 'val' for validation set
    imgsz=6016, # même valeur celle utilisée dans le code https://github.com/jodivaso/varrodetector/blob/main/varroa_mite_gui.py#L2507
    batch=4,
    conf=0.1,  # Seuil de confiance
    iou=0.5,
    max_det=2000,
    save_json=True,
    save=True,
    show_labels=False,
    show_conf=False,
    line_width=2
)

# Afficher les résultats
# Print results
print(f"Precision: {results.box.p}")
print(f"Recall: {results.box.r}")
print(f"mAP50: {results.box.map50}")
print(f"mAP50-95: {results.box.map}")

Ultralytics 8.3.249  Python-3.12.7 torch-2.9.1+cpu CPU (Intel Core(TM) i7-10510U 1.80GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 632.795.6 MB/s, size: 1693.7 KB)
val: Scanning D:\dev\ia\cours\varroacounter\varroa-counter-large-3\train\labels.cache... 19 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 19/19  0.0s
WARNING Box and segment counts should be equal, but got len(segments) = 418, len(boxes) = 1595. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 29.9s/it 2:2940.4s2
                   all         19       1595      0.414      0.401      0.352     0.0985
Speed: 124.8ms preprocess, 6928.0ms inference, 0.0ms loss, 14.1ms postprocess per image
Saving D:\dev\

*En résumé la qualité de ce modèle par rapport à mon problème est faible*

**Precision : 0.4135 (41%)**
Sur toutes les détections faites par le modèle, seulement 41% sont correctes. Cela signifie que ~59% des détections sont des faux positifs (le modèle détecte un varroa là où il n'y en a pas).

**Recall : 0.4013 (40%)**
Sur tous les varroas réellement présents dans les images, le modèle n'en détecte que 40%. Il rate donc ~60% des varroas réels.

**mAP50 : 0.352**
La précision moyenne (mean Average Precision) avec un seuil IoU de 50%. C'est la métrique standard de performance en détection d'objets. Un score de 0.35 est faible.

**mAP50-95 : 0.098**
La même métrique mais moyennée sur des seuils IoU de 50% à 95% (plus strict sur la précision de localisation). Un score de ~0.10 est très faible, ce qui indique que même quand le modèle trouve un varroa, la boîte englobante est souvent mal positionnée.

Je décide donc de faire un test avec 1 seule image afin de déterminer plus précisément où le problème se trouve

In [ ]:

from ultralytics import YOLO

# Charger le modèle entraîné
model = YOLO('model_mdpi_3291496/weights/best.pt')

# Effectuer la détection sur l'image
results = model.predict(
    source="varroa-counter-large-3/train/images/IMG_0223_cropped_jpg.rf.0e4c7399e5f9e2e6287c52cd865bae46.jpg",
    imgsz=6016,
    max_det=2000,
    conf=0.1,
    iou=0.5,
    save=True,
    show_labels=False,
    show_conf=False,
    line_width=2
)

print(f"Varroas détectés: {len(results[0].boxes)}")


image 1/1 d:\dev\ia\cours\varroacounter\varroa-counter-large-3\train\images\IMG_0223_cropped_jpg.rf.0e4c7399e5f9e2e6287c52cd865bae46.jpg: 6016x4544 250 varroas, 4753.5ms
Speed: 343.0ms preprocess, 4753.5ms inference, 27.0ms postprocess per image at shape (1, 3, 6016, 4544)
Results saved to D:\dev\runs\detect\predict16
Varroas détectés: 250


Voici un zoom sur l'image générée par la détection
![image annoté après prédiction](https://github.com/geda/varroacounter/blob/main/IMG_0223_cropped_predict_16_zoom.jpg?raw=1)

On constate les problèmes suivants:
- le modèle confond les gouttes d'eau et les varroas
- le modèle oublie des varroas pourtant bien visibles




In [ ]:
!pip install ultralytics
# import the needed librairies
import ultralytics

from ultralytics import YOLO
# entrainement du modèle
# Je créer un nouveau modèle depuis zéro
model = YOLO('varroacounter/model_mdpi_3291496/weights/best.pt')

# lancement de l'entraînement avec seulement 1 seul passage et en abaissant le nombre de batch (nombre d'images traitées simultanément).
# le but est de tester si les capacitées de ma machine sont suffisantes pour entraîner mon modèle
results = model.train(data="varroa-counter-large-3/data.yaml",
                      epochs=1,
                      imgsz=6016,
                      batch=16,
                      freeze=10,
                      max_det=2000,
                      conf=0.1,
                      iou = 0.5)
print (results)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.12 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.1, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=varroa-counter-large-3/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=

Je décide de faire une prédiction valider le modèle avec uniquement les images provenant du site espagnol. Je ne sais pas si elles ont été utilisées lors de l'entraînement du modèle, mais décide de faire une comparaison.

In [ ]:
import os
import shutil
import yaml
from pathlib import Path


def create_filtered_dataset(source_dir, dest_dir, prefix="IMG_6"):
    """
    Crée une copie d'un dataset YOLO en ne gardant que les images
    dont le nom commence par le préfixe spécifié.

    Args:
        source_dir: Chemin du dataset source (ex: "varroa-counter-v3-3")
        dest_dir: Chemin du dataset destination
        prefix: Préfixe des images à conserver (ex: "IMG_6")
    """
    source_dir = Path(source_dir)
    dest_dir = Path(dest_dir)

    # Copier data.yaml et fichiers README
    dest_dir.mkdir(parents=True, exist_ok=True)
    for f in source_dir.glob("*.yaml"):
        shutil.copy2(f, dest_dir / f.name)
    for f in source_dir.glob("*.txt"):
        shutil.copy2(f, dest_dir / f.name)

    total_copied = 0

    # Parcourir les splits (train, valid, test)
    for split_dir in source_dir.iterdir():
        if not split_dir.is_dir():
            continue

        images_dir = split_dir / "images"
        labels_dir = split_dir / "labels"

        if not images_dir.exists():
            continue

        # Créer les dossiers de destination
        dest_images = dest_dir / split_dir.name / "images"
        dest_labels = dest_dir / split_dir.name / "labels"
        dest_images.mkdir(parents=True, exist_ok=True)
        dest_labels.mkdir(parents=True, exist_ok=True)

        # Copier les images correspondant au préfixe et leurs labels
        for img_file in images_dir.iterdir():
            if img_file.name.startswith(prefix):
                shutil.copy2(img_file, dest_images / img_file.name)

                # Copier le label correspondant (.txt)
                label_file = labels_dir / (img_file.stem + ".txt")
                if label_file.exists():
                    shutil.copy2(label_file, dest_labels / label_file.name)

                total_copied += 1

    print(f"Dataset filtré créé dans: {dest_dir}")
    print(f"Images copiées: {total_copied}")

    # Lister le contenu
    for split_dir in sorted(dest_dir.iterdir()):
        if split_dir.is_dir():
            imgs = list((split_dir / "images").glob("*"))
            print(f"  {split_dir.name}: {len(imgs)} images")


# Créer le dataset filtré avec uniquement les images IMG_6xxx
create_filtered_dataset(
    source_dir="varroa-counter-v3-3",
    dest_dir="varroa-counter-v3-3-IMG6",
    prefix="IMG_6"
)

Dataset filtré créé dans: varroa-counter-v3-3-IMG6
Images copiées: 5
  test: 5 images
  valid: 0 images


In [ ]:
from ultralytics import YOLO

# Charger le modèle entraîné
model = YOLO('model_mdpi_3291496/weights/best.pt')

# Effectuer la détection sur l'image
results = model.val(
    data="varroa-counter-v3-3-IMG6/data.yaml",
    split='test',  # or 'val' for validation set
    imgsz=6016, # même valeur celle utilisée dans le code https://github.com/jodivaso/varrodetector/blob/main/varroa_mite_gui.py#L2507
    batch=16,
    conf=0.1,  # Seuil de confiance
    iou=0.5,
    max_det=2000,
    save_json=True,
    save=True,
    show_labels=False,
    show_conf=False,
    line_width=2
)

# Afficher les résultats
# Print results
print(f"Precision: {results.box.p}")
print(f"Recall: {results.box.r}")
print(f"mAP50: {results.box.map50}")
print(f"mAP50-95: {results.box.map}")


Ultralytics 8.3.249  Python-3.12.7 torch-2.9.1+cpu CPU (Intel Core(TM) i7-10510U 1.80GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 85.96.4 MB/s, size: 1745.4 KB)
val: Scanning D:\dev\ia\cours\varroacounter\varroa-counter-v3-3-IMG6\test\labels... 5 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 5/5 118.5it/s 0.0s
val: New cache created: D:\dev\ia\cours\varroacounter\varroa-counter-v3-3-IMG6\test\labels.cache
WARNING Box and segment counts should be equal, but got len(segments) = 209, len(boxes) = 745. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 123.3s/it 2:03
                   all          5        745      0.609      0.483      0.483      0.148
Speed: 890.8

Ces résultats montrent une performance améliorée par rapport à la validation du set complet, mais quand-même une performance relativement faible.

Precision (0.61) — Sur toutes les détections faites, 61% sont correctes. Environ 4 détections sur 10 sont des faux positifs.

Recall (0.48) — Le modèle ne détecte que 48% des varroas réellement présents. Il en rate plus de la moitié.

mAP50 (0.48) — Performance globale à IoU 50% insuffisante. Un modèle correct vise >0.5, un bon modèle >0.7.

mAP50-95 (0.15) — La localisation précise est très faible. Même quand le modèle trouve un varroa, la bounding box est souvent mal positionnée.

In [ ]:
from ultralytics import YOLO

# Charger le modèle entraîné
model = YOLO('model_mdpi_3291496/weights/best.pt')

# Effectuer la détection sur l'image
results = model.predict(
    source="varroas/Sample images/IMG_6187.jpg",
    imgsz=6016,
    max_det=2000,
    conf=0.1,
    iou=0.5,
    save=True,
    show_labels=False,
    show_conf=False,
    line_width=2
)

print(f"Varroas détectés: {len(results[0].boxes)}")


image 1/1 d:\dev\ia\cours\varroacounter\varroas\Sample images\IMG_6187.jpg: 6016x4512 85 varroas, 7622.3ms
Speed: 525.4ms preprocess, 7622.3ms inference, 28.9ms postprocess per image at shape (1, 3, 6016, 4512)
Results saved to D:\dev\runs\detect\predict15
Varroas détectés: 85


On constate sur l'image résultant de la prédiction les problèmes suivants:
- lorsque le varroa est posé sur de la cire, il est mal détecté
- lorsque 2 varroas sont côte à côte, parfois un seul varroa est détecté

![image de l'étude annoté](https://github.com/geda/varroacounter/blob/main/IMG_6187_predict_15_zoom.jpg?raw=1)


In [ ]:
!pip install sahi

from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",  # Compatible YOLO11
    model_path="model_mdpi_3291496/weights/best.pt",
    confidence_threshold=0.1
)

result = get_sliced_prediction(
    image="varroa-counter-v3-3/test/images/IMG_0223_jpg.rf.a0db71f36b6b62fd38a1deca87b0a1f0.jpg",
    detection_model=model,
    slice_height=1024,
    slice_width=1024,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2
)

result.export_visuals(export_dir="results/")


Performing prediction on 35 slices.


In [ ]:
from ultralytics import YOLO

model = YOLO('model_mdpi_3291496/weights/best.pt')
results = model.train(
    data="varroa-counter-v3-2/data.yaml",
    epochs=150,
    imgsz=2048,
    batch=4,
    freeze=10  # Gèle les premières couches, ajuste seulement les dernières
)

In [ ]:
# 2. Tester le modèle fine-tuné
# Le meilleur modèle est automatiquement sauvegardé dans runs/detect/trainX/weights/best.pt
model_finetuned = YOLO('runs/train23/weights/best.pt')

# Validation sur le set de test
results = model_finetuned.val(
    data="varroa-counter-v3-3/data.yaml",
    split='test',  # ou 'val'
    imgsz=2048,
    conf=0.1,
    iou=0.5,
    max_det=2000,
    save=True
)

print(f"Precision: {results.box.p}")
print(f"Recall: {results.box.r}")
print(f"mAP50: {results.box.map50}")
print(f"mAP50-95: {results.box.map}")

Ultralytics 8.3.249  Python-3.12.7 torch-2.9.1+cpu CPU (Intel Core(TM) i7-10510U 1.80GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 138.130.7 MB/s, size: 1597.2 KB)
val: Scanning D:\dev\ia\cours\varroacounter\varroa-counter-v3-3\test\labels... 32 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 32/32 228.3it/s 0.1s.2s
val: New cache created: D:\dev\ia\cours\varroacounter\varroa-counter-v3-3\test\labels.cache
WARNING Box and segment counts should be equal, but got len(segments) = 209, len(boxes) = 1057. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 14.3s/it 28.6s49.4s
                   all         32       1057      0.566      0.459       0.51      0.268
Speed: 1

In [ ]:
# Test du modèle avec le dataset Varroa-board-1
from ultralytics import YOLO

model = YOLO('model_mdpi_3291496/weights/best.pt')

results = model.val(
    data="Varroa-board-1/data.yaml",
    split='test',  # Utiliser le split 'test'
    imgsz=(6016), max_det=2000, conf=0.1, iou = 0.5,
    save=True, show_labels=False, line_width=2, save_txt=True, save_conf=True
)

print("\nRésultats de validation sur Varroa-board-1:")
print(f"Précision: {results.box.p}")
print(f"Recall: {results.box.r}")
print(f"mAP50: {results.box.map50}")
print(f"mAP50-95: {results.box.map}")

#Résultats de validation sur Varroa-board-1:
#Précision: [    0.80237]
#Recall: [    0.42177]
#mAP50: 0.584942355545071
#mAP50-95: 0.2293648037166341

Ultralytics 8.3.249  Python-3.12.7 torch-2.9.1+cpu CPU (Intel Core(TM) i7-10510U 1.80GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


FileNotFoundError: [34m[1mval: [0mError loading data from None
See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [ ]:
# Installation de SAHI pour la détection par découpage
!pip install sahi

In [ ]:
# Détection avec SAHI (Slicing Aided Hyper Inference)
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from sahi.utils.cv import read_image
import os

# Charger le modèle via SAHI
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",  # Compatible avec YOLO11
    model_path="model_mdpi_3291496/weights/best.pt",
    confidence_threshold=0.1,
    device="cpu"
)

# Image à analyser
image_path = 'varroa-counter-v3-3/test/images/IMG_6098_jpg.rf.eb517a2863fbb704a84afce55b287455.jpg'

# Effectuer la prédiction par découpage
result = get_sliced_prediction(
    image_path,
    detection_model,
    slice_height=4096,
    slice_width=4096,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
    postprocess_type="NMS",
    postprocess_match_metric="IOU",
    postprocess_match_threshold=0.5
)

# Créer le dossier de résultats
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

# Exporter l'image avec SEULEMENT les bounding boxes (sans texte de classe)
result.export_visuals(
    export_dir=output_dir,
    file_name="detection_result",
    hide_labels=True,      # Masque le texte de classe
    hide_conf=True,        # Masque le score de confiance
    rect_th=1              # Épaisseur des rectangles
)

print(f"Image exportée dans: {output_dir}/detection_result.png")

# ============================================================
# INFORMATIONS DÉTAILLÉES SUR LES RÉSULTATS
# ============================================================

print(f"\n{'='*70}")
print("RÉSUMÉ DE LA DÉTECTION")
print(f"{'='*70}")

object_predictions = result.object_prediction_list
print(f"\nNombre total de varroas détectés: {len(object_predictions)}")

# Statistiques sur les scores de confiance
if object_predictions:
    confidences = [pred.score.value for pred in object_predictions]
    print(f"\nStatistiques de confiance:")
    print(f"  - Confiance moyenne: {sum(confidences)/len(confidences):.3f}")
    print(f"  - Confiance min: {min(confidences):.3f}")
    print(f"  - Confiance max: {max(confidences):.3f}")

    # Distribution par tranches de confiance
    print(f"\nDistribution par niveau de confiance:")
    ranges = [(0.1, 0.3), (0.3, 0.5), (0.5, 0.7), (0.7, 0.9), (0.9, 1.0)]
    for low, high in ranges:
        count = sum(1 for c in confidences if low <= c < high)
        print(f"  - {low:.1f} - {high:.1f}: {count} détections")

    # Statistiques sur les tailles des bounding boxes
    areas = []
    for pred in object_predictions:
        bbox = pred.bbox
        width = bbox.maxx - bbox.minx
        height = bbox.maxy - bbox.miny
        areas.append(width * height)

    print(f"\nStatistiques des bounding boxes (en pixels):")
    print(f"  - Aire moyenne: {sum(areas)/len(areas):.1f} px²")
    print(f"  - Aire min: {min(areas):.1f} px²")
    print(f"  - Aire max: {max(areas):.1f} px²")

# Afficher les détails des 10 premières détections
print(f"\n{'='*70}")
print("DÉTAILS DES 10 PREMIÈRES DÉTECTIONS")
print(f"{'='*70}")

for i, pred in enumerate(object_predictions[:10]):
    bbox = pred.bbox
    print(f"\nDétection #{i+1}:")
    print(f"  - Classe: {pred.category.name}")
    print(f"  - Confiance: {pred.score.value:.3f}")
    print(f"  - Bounding box: x1={bbox.minx:.0f}, y1={bbox.miny:.0f}, x2={bbox.maxx:.0f}, y2={bbox.maxy:.0f}")
    print(f"  - Dimensions: {bbox.maxx - bbox.minx:.0f}x{bbox.maxy - bbox.miny:.0f} pixels")
    print(f"  - Centre: ({(bbox.minx + bbox.maxx)/2:.0f}, {(bbox.miny + bbox.maxy)/2:.0f})")

Performing prediction on 6 slices.
Image exportée dans: results/detection_result.png

RÉSUMÉ DE LA DÉTECTION

Nombre total de varroas détectés: 245

Statistiques de confiance:
  - Confiance moyenne: 0.629
  - Confiance min: 0.101
  - Confiance max: 0.939

Distribution par niveau de confiance:
  - 0.1 - 0.3: 61 détections
  - 0.3 - 0.5: 26 détections
  - 0.5 - 0.7: 15 détections
  - 0.7 - 0.9: 98 détections
  - 0.9 - 1.0: 45 détections

Statistiques des bounding boxes (en pixels):
  - Aire moyenne: 3428.8 px²
  - Aire min: 294.6 px²
  - Aire max: 8172.9 px²

DÉTAILS DES 10 PREMIÈRES DÉTECTIONS

Détection #1:
  - Classe: varroa
  - Confiance: 0.939
  - Bounding box: x1=2194, y1=3837, x2=2257, y2=3906
  - Dimensions: 63x69 pixels
  - Centre: (2226, 3871)

Détection #2:
  - Classe: varroa
  - Confiance: 0.932
  - Bounding box: x1=1761, y1=1243, x2=1821, y2=1311
  - Dimensions: 60x68 pixels
  - Centre: (1791, 1277)

Détection #3:
  - Classe: varroa
  - Confiance: 0.925
  - Bounding box: x1=

In [ ]:
# Exporter les résultats détaillés en CSV
import csv

csv_path = os.path.join(output_dir, "detections.csv")

with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'classe', 'confiance', 'x1', 'y1', 'x2', 'y2', 'largeur', 'hauteur', 'aire', 'centre_x', 'centre_y'])

    for i, pred in enumerate(object_predictions):
        bbox = pred.bbox
        width = bbox.maxx - bbox.minx
        height = bbox.maxy - bbox.miny
        area = width * height
        center_x = (bbox.minx + bbox.maxx) / 2
        center_y = (bbox.miny + bbox.maxy) / 2

        writer.writerow([
            i + 1,
            pred.category.name,
            f"{pred.score.value:.4f}",
            f"{bbox.minx:.1f}",
            f"{bbox.miny:.1f}",
            f"{bbox.maxx:.1f}",
            f"{bbox.maxy:.1f}",
            f"{width:.1f}",
            f"{height:.1f}",
            f"{area:.1f}",
            f"{center_x:.1f}",
            f"{center_y:.1f}"
        ])

print(f"Résultats exportés en CSV: {csv_path}")
print(f"\nLe fichier contient {len(object_predictions)} détections avec:")
print("  - ID unique")
print("  - Classe détectée")
print("  - Score de confiance")
print("  - Coordonnées du bounding box (x1, y1, x2, y2)")
print("  - Dimensions (largeur, hauteur)")
print("  - Aire en pixels²")
print("  - Coordonnées du centre")

In [ ]:
# Test avec découpage d'image en 4 parties
from ultralytics import YOLO
from PIL import Image
import os

# Charger le modèle
model = YOLO('model_mdpi_3291496/weights/best.pt')

# Chemin de l'image à tester
image_path = 'Varroa-board-1/test/images/IMG_0226_jpg.rf.c97161f83bb98300231bd6318d7dee3b.jpg'

# Charger l'image
img = Image.open(image_path)
width, height = img.size
print(f"Taille originale de l'image: {width}x{height}")

# Créer un dossier pour les images découpées
output_dir = 'temp_tiles'
os.makedirs(output_dir, exist_ok=True)

# Découper l'image en 4 parties (2x2)
half_width = width // 2
half_height = height // 2

tiles = []
positions = [
    (0, 0, half_width, half_height, "top_left"),
    (half_width, 0, width, half_height, "top_right"),
    (0, half_height, half_width, height, "bottom_left"),
    (half_width, half_height, width, height, "bottom_right")
]

# Découper et sauvegarder chaque partie
for i, (x1, y1, x2, y2, name) in enumerate(positions):
    tile = img.crop((x1, y1, x2, y2))
    tile_path = os.path.join(output_dir, f'tile_{i+1}_{name}.jpg')
    tile.save(tile_path)
    tiles.append((tile_path, name, x1, y1))
    print(f"Partie {i+1} ({name}): {tile.size[0]}x{tile.size[1]} sauvegardée")

print(f"\n{'='*60}")
print("DÉTECTION SUR CHAQUE PARTIE")
print(f"{'='*60}\n")

# Tester le modèle sur chaque partie
total_detections = 0
all_results = []

for tile_path, name, offset_x, offset_y in tiles:
    print(f"\n--- Détection sur {name} ---")
    results = model.predict(
        source=tile_path,
   imgsz=(6016), max_det=2000, conf=0.1, iou = 0.5,
    save=True, show_labels=False, line_width=2, save_txt=True, save_conf=True
    )

    detections = len(results[0].boxes)
    total_detections += detections
    all_results.append((name, detections, results[0]))

    print(f"Varroas détectés: {detections}")

print(f"\n{'='*60}")
print(f"RÉSUMÉ DES DÉTECTIONS")
print(f"{'='*60}")
print(f"\nNombre total de varroas détectés: {total_detections}")
print("\nDétail par partie:")
for name, count, _ in all_results:
    print(f"  - {name:15s}: {count:3d} varroas")

print(f"\nImages découpées sauvegardées dans: {output_dir}/")
print(f"Résultats de détection sauvegardés dans: runs/detect/predict*/")


FileNotFoundError: [Errno 2] No such file or directory: 'Varroa-board-1/test/images/IMG_0226_jpg.rf.c97161f83bb98300231bd6318d7dee3b.jpg'

In [ ]:
import os
import shutil
import yaml
from pathlib import Path


def create_filtered_dataset(source_dir, dest_dir, prefix="IMG_6"):
    """
    Crée une copie d'un dataset YOLO en ne gardant que les images
    dont le nom commence par le préfixe spécifié.

    Args:
        source_dir: Chemin du dataset source (ex: "varroa-counter-v3-3")
        dest_dir: Chemin du dataset destination
        prefix: Préfixe des images à conserver (ex: "IMG_6")
    """
    source_dir = Path(source_dir)
    dest_dir = Path(dest_dir)

    # Copier data.yaml et fichiers README
    dest_dir.mkdir(parents=True, exist_ok=True)
    for f in source_dir.glob("*.yaml"):
        shutil.copy2(f, dest_dir / f.name)
    for f in source_dir.glob("*.txt"):
        shutil.copy2(f, dest_dir / f.name)

    total_copied = 0

    # Parcourir les splits (train, valid, test)
    for split_dir in source_dir.iterdir():
        if not split_dir.is_dir():
            continue

        images_dir = split_dir / "images"
        labels_dir = split_dir / "labels"

        if not images_dir.exists():
            continue

        # Créer les dossiers de destination
        dest_images = dest_dir / split_dir.name / "images"
        dest_labels = dest_dir / split_dir.name / "labels"
        dest_images.mkdir(parents=True, exist_ok=True)
        dest_labels.mkdir(parents=True, exist_ok=True)

        # Copier les images correspondant au préfixe et leurs labels
        for img_file in images_dir.iterdir():
            if img_file.name.startswith(prefix):
                shutil.copy2(img_file, dest_images / img_file.name)

                # Copier le label correspondant (.txt)
                label_file = labels_dir / (img_file.stem + ".txt")
                if label_file.exists():
                    shutil.copy2(label_file, dest_labels / label_file.name)

                total_copied += 1

    print(f"Dataset filtré créé dans: {dest_dir}")
    print(f"Images copiées: {total_copied}")

    # Lister le contenu
    for split_dir in sorted(dest_dir.iterdir()):
        if split_dir.is_dir():
            imgs = list((split_dir / "images").glob("*"))
            print(f"  {split_dir.name}: {len(imgs)} images")


# Créer le dataset filtré avec uniquement les images IMG_6xxx
create_filtered_dataset(
    source_dir="varroa-counter-v3-3",
    dest_dir="varroa-counter-v3-3-IMG6",
    prefix="IMG_6"
)